In [298]:
import pandas as pd
import numpy as np
import random, math

In [299]:
random.seed(786)
np.random.seed(786)

In [300]:
df = pd.read_csv('final_data.csv')
# df1 = df[['compound','neg','neu','pos','Close']]
df1 = df[['compound','neg','neu','pos']]
df2=df[['Close']]
# df = df[['neg','neu','pos','Close']]

In [301]:
from sklearn import preprocessing
min_max_scalar=preprocessing.MinMaxScaler()
data=min_max_scalar.fit_transform(df1)
# X_test=min_max_scalar.fit_transform(X_test)
# Y_train=min_max_scalar.fit_transform(Y_train)
# Y_test=min_max_scalar.fit_transform(Y_test)

In [302]:
df1=pd.DataFrame(data)

In [303]:
df1.head()

,0,1,2,3
0,0.053189,0.494253,0.800429,0.341667
1,0.150989,0.382184,0.864807,0.250000
2,0.027553,0.916667,0.684549,0.183333
3,0.105067,0.514368,0.778970,0.391667
4,0.208367,0.402299,0.821888,0.391667


In [304]:
close_max=df2.max()
df2/=close_max
# df2=np.array(df2)

In [305]:
df2.head()

,Close
0,0.387238
1,0.394951
2,0.404868
3,0.405030
4,0.405062


In [306]:
df1['4']=df2

In [307]:
df1.head()

,0,1,2,3,4
0,0.053189,0.494253,0.800429,0.341667,0.387238
1,0.150989,0.382184,0.864807,0.250000,0.394951
2,0.027553,0.916667,0.684549,0.183333,0.404868
3,0.105067,0.514368,0.778970,0.391667,0.405030
4,0.208367,0.402299,0.821888,0.391667,0.405062


In [308]:
# df['Close']=df['Close']/400

In [309]:
data = df1.values.tolist()

In [310]:
data[:2]

[[0.05318934194590236,
  0.4942528735632184,
  0.8004291845493562,
  0.3416666666666667,
  0.3872379038791846],
 [0.1509890997174001,
  0.38218390804597707,
  0.8648068669527895,
  0.25,
  0.39495090255047477]]

In [311]:
# data = data.tolist()

In [312]:
inner_threshold = 0.005;
rate_var = 0;

final_ansr = []

In [313]:
sigmoid=lambda x:1/(1+math.e**(-x))

In [314]:
def fit_func(rec, i):
    x1, x2, x3, x4, y = i[0], i[1], i[2], i[3], i[4]

    out5 = (x1 * rec['data']['w15']) + (x2 * rec['data']['w25']) + (x3 * rec['data']['w35']) + (x4 * rec['data']['w45']) + rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (x1 * rec['data']['w16']) + (x2 * rec['data']['w26']) + (x3 * rec['data']['w36']) + (x4 * rec['data']['w46']) + rec['data']['o6']
    out6 = sigmoid(out6)
    
    out7 = (x1 * rec['data']['w17']) + (x2 * rec['data']['w27']) + (x3 * rec['data']['w37']) + (x4 * rec['data']['w47']) + rec['data']['o7']
    out7 = sigmoid(out7)

    out8 = (out5 * rec['data']['w58']) + (out6 * rec['data']['w68']) + (out7 * rec['data']['w78']) + rec['data']['o8']
    out8 = sigmoid(out8)
    
    out9 = (out5 * rec['data']['w59']) + (out6 * rec['data']['w69']) + (out7 * rec['data']['w79']) + rec['data']['o9']
    out9 = sigmoid(out9)

    out10 = (out8 * rec['data']['w810']) + (out9 * rec['data']['w910']) + rec['data']['o10']
    out10 = sigmoid(out10)

    err10 = out10 * (1 - out10) * (y - out10)

    if abs(err10) <= inner_threshold:
        # final_ansr.append(out6 * 20000)
        rec['fitness'] += 1


In [315]:
nList = [];
m = 20;
n = 25;
mutation_prob = 75;
mutation = 0.5;

In [316]:
# ranges for weights & biases
wl = -2;
wr = 2;
bl = -1;
br = 1

In [317]:
total_weights=20
total_biases=6

In [318]:
for i in range(n):
    # floating random weights
    w15, w16, w17, w25, w26,w27, w35, w36,w37, w45, w46,w47, w58,w59,w68,w69,w78,w79, w810, w910 = np.random.uniform(wl, wr, total_weights)
    # floating random biases
    o5, o6, o7,o8, o9, o10 = np.random.uniform(bl, br, total_biases)

    nList.append({'data': {'w15': w15, 'w16': w16, 'w17': w17, 'w25': w25, 'w26': w26, 'w27': w27,
                           'w35': w35,'w36': w36, 'w37': w37, 'w45': w45, 'w46': w46,
                           'w47': w47, 'w58': w58, 'w59': w59, 'w68': w68, 'w69': w69, 'w78': w78,
                           'w79': w79,'w810': w810,'w910': w910, 'o5': o5, 'o6': o6, 'o7': o7,
                           'o8': o8, 'o9': o9, 'o10': o10},
                           'fitness': 0})

In [319]:
# fitness calculation for first time/initial population
for rec in nList:
    for i in data:
        fit_func(rec, i)
        
    print(rec['fitness'])

188
242
0
153
73
97
382
71
57
8
254
383
48
176
248
247
149
144
181
169
46
8
81
93
0


In [320]:
count = 0;
final_rec = 0
flag = False
stop_at=50

In [321]:
while (1):
    count += 1
    mList = []

    print(f"gen: {count}")

    if count > stop_at:
        final_rec = sorted(nList, key=lambda f: f['fitness'], reverse=True)[0]
        break

    rec_num = 0
    for rec in nList:
        # print("rec_index: {}".format(rec_num))
        rec_num += 1
        if rec['fitness'] == len(data):
            print("rec_index: {}".format(rec_num))
            final_rec = rec
            # print('gen: {}'.format(count))
            print('Answer: {}'.format(rec))
            flag = True
            break

    if flag:
        break

    for j in range(int(m / 2)):
        # randomly parent selection(not same)
        # t=temp

        t = random.sample(range(n), 2)

        p1 = nList[t[0]]
        p2 = nList[t[1]]

        # crossover
        crossover_point=int((total_weights+total_biases)/2)
        ch1 = list(p1['data'].values())[:crossover_point] + list(p2['data'].values())[crossover_point:]
        ch2 = list(p2['data'].values())[:crossover_point] + list(p1['data'].values())[crossover_point:]

        # mutation work
        for k in [ch1, ch2]:
            if random.randint(0, 100) <= mutation_prob:
                # get random for mutation in weights
                t = random.sample(range(total_weights), 3)
                # get random for mutation in bias
                t1 = random.randint(total_weights, (total_weights+total_biases)-1)
                # odd,do +ve
                if random.randint(1, 2) == 1:
                    # mutation in weights
                    if (k[t[0]] + mutation) >= wl and (k[t[0]] + mutation) <= wr:
                        k[t[0]] += mutation

                    if (k[t[1]] + mutation) >= wl and (k[t[1]] + mutation) <= wr:
                        k[t[1]] += mutation

                    if (k[t[2]] + mutation) >= wl and (k[t[2]] + mutation) <= wr:
                        k[t[2]] += mutation

                    # mutation in bias
                    if (k[t1] + mutation) >= bl and (k[t1] + mutation) <= br:
                        k[t1] += mutation

                # even,do -ve
                else:
                    # mutation in weights
                    if (k[t[0]] - mutation) >= wl and (k[t[0]] - mutation) <= wr:
                        k[t[0]] -= mutation

                    if (k[t[1]] - mutation) >= wl and (k[t[1]] - mutation) <= wr:
                        k[t[1]] -= mutation

                    if (k[t[2]] - mutation) >= wl and (k[t[2]] - mutation) <= wr:
                        k[t[2]] -= mutation

                    # mutation in bias
                    if (k[t1] - mutation) >= bl and (k[t1] - mutation) <= br:
                        k[t1] -= mutation

                        
        ch1 = {'data': {'w15': ch1[0], 'w16': ch1[1], 'w17': ch1[2], 'w25': ch1[3], 'w26': ch1[4], 'w27': ch1[5],
                        'w35': ch1[6],
                        'w36': ch1[7], 'w37': ch1[8], 'w45': ch1[9], 'w46': ch1[10],
                        'w47': ch1[11], 'w58': ch1[12], 'w59': ch1[13], 'w68': ch1[14], 'w69': ch1[15], 'w78': ch1[16],
                        'w79': ch1[17],
                        'w810': ch1[18], 'w910': ch1[19], 'o5': ch1[20], 'o6': ch1[21],
                        'o7': ch1[22], 'o8': ch1[23], 'o9': ch1[24], 'o10': ch1[25]}, 'fitness': 0}

        ch2 = {'data': {'w15': ch2[0], 'w16': ch2[1], 'w17': ch2[2], 'w25': ch2[3], 'w26': ch2[4], 'w27': ch2[5],
                        'w35': ch2[6],
                        'w36': ch2[7], 'w37': ch2[8], 'w45': ch2[9], 'w46': ch2[10],
                        'w47': ch2[11], 'w58': ch2[12], 'w59': ch2[13], 'w68': ch2[14], 'w69': ch2[15], 'w78': ch2[16],
                        'w79': ch2[17],
                        'w810': ch2[18], 'w910': ch2[19], 'o5': ch2[20], 'o6': ch2[21],
                        'o7': ch2[22], 'o8': ch2[23], 'o9': ch2[24], 'o10': ch2[25]}, 'fitness': 0}


        # calculate fitness of children
        for rec in [ch1, ch2]:
            for i in data:
                fit_func(rec, i)

        mList.append(ch1)
        mList.append(ch2)

    # combine both mList and nList and sort it with respect fit func value,also ovverides the nList,so we used in next iteration

    nList = sorted(nList + mList, key=lambda item: item['fitness'], reverse=True)[:25]


gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30
gen: 31
gen: 32
gen: 33
gen: 34
gen: 35
gen: 36
gen: 37
gen: 38
gen: 39
gen: 40
gen: 41
gen: 42
gen: 43
gen: 44
gen: 45
gen: 46
gen: 47
gen: 48
gen: 49
gen: 50
gen: 51


In [322]:
for i in data:
    x1, x2, x3, x4, y = i[0], i[1], i[2], i[3], i[4]

    out5 = (x1 * rec['data']['w15']) + (x2 * rec['data']['w25']) + (x3 * rec['data']['w35']) + (x4 * rec['data']['w45']) + rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (x1 * rec['data']['w16']) + (x2 * rec['data']['w26']) + (x3 * rec['data']['w36']) + (x4 * rec['data']['w46']) + rec['data']['o6']
    out6 = sigmoid(out6)
    
    out7 = (x1 * rec['data']['w17']) + (x2 * rec['data']['w27']) + (x3 * rec['data']['w37']) + (x4 * rec['data']['w47']) + rec['data']['o7']
    out7 = sigmoid(out7)

    out8 = (out5 * rec['data']['w58']) + (out6 * rec['data']['w68']) + (out7 * rec['data']['w78']) + rec['data']['o8']
    out8 = sigmoid(out8)
    
    out9 = (out5 * rec['data']['w59']) + (out6 * rec['data']['w69']) + (out7 * rec['data']['w79']) + rec['data']['o9']
    out9 = sigmoid(out9)

    out10 = (out8 * rec['data']['w810']) + (out9 * rec['data']['w910']) + rec['data']['o10']
    out10 = sigmoid(out10)

    final_ansr.append(out10*close_max)

In [323]:
# final_ansr=final_ansr[-len(data):]
df['Y-ENN'] = pd.DataFrame(final_ansr)
df.to_csv('NN-DATA2.csv', index=False)

In [324]:
from sklearn.metrics import r2_score
r2_score(df[['Close']],df['Y-ENN'])

-0.8684247040915436

In [325]:
df[['Close']].head()

,Close
0,119.49
1,121.87
2,124.93
3,124.98
4,124.99


In [326]:
df['Y-ENN'].head()

0    125.122705
1    125.165799
2    125.113312
3    125.115474
4    125.136513
Name: Y-ENN, dtype: float64

In [327]:
0    140.936089
1    140.920043
2    140.903992
3    140.966967
4    140.986424
Name: Y-ENN, dtype: float64

SyntaxError: invalid syntax (<ipython-input-327-3e53db0691e5>, line 1)